# Imports

In [1]:
%matplotlib ipympl

In [2]:
import numpy as np
import math
import scipy
import py3nj
import pandas as pd
import matplotlib.pyplot as plt

# Finding the Hamiltonian

The basic Hamiltonian is given by the kinetic energy of a rigid rotor, plus the interaction of the dipole with the electric field.

$$ H = \frac{L^2}{2I} + \vec{d}\cdot\vec{E} $$

Now we will include the nuclear spin-orbital angular momentum coupling term, as well as the dipole-dipole interaction term.

$$ H_{tot} = \frac{L^2}{2I} + \vec{d}\cdot\vec{E} + H_{JI} + H_{dip} $$

Now we have to include a nuclear spin eigenfunction in the eigenfunctions for the Hamiltonian. We find each element in the hamiltonian as follows:

$$ H_{i j} (m)= \braket{m m_I|H| m' m_I'}$$

Where m is the projection of angular momentum J onto the Z-axis, and m_I is the projection of the nuclear spin onto the Z-axis. Note that we will compute the energies for fixed J and I values, with eigenstates labeled $\ket{m \ m_I}$. As m $\in [-J,J]$ and $m_I \in [-I,I]$, the dimension of our hamiltonian will be (2J+1)(2I+1). We will diagonalize the Hamiltonian matrix to find its eigenvalues, yielding the new energies of the system.

Note that for the order of the eigenstates, we will use the counting scheme as follows,

For J = 1 and I = 1 dim(H) = 9 and $\psi _0 = \ket{-1 \ -1}$, $\psi _1 = \ket{-1 \ 0}$, $\psi _2 = \ket{-1 \ 1}$, ... , $\psi _7 = \ket{1 \ 0}$, $\psi _8 = \ket{1 \ 1}$

In [40]:
def getHij(index_1, index_2, E, J=1, I=1, B0=1, hbar=1, dip=1, C_I=1):
    num_m = 2*J+1
    num_mI = 2*I+1
    dim_H = num_m * num_mI
    
    if index_1 < 0 or index_2 < 0 or index_1 >= dim_H or index_2 >= dim_H:
        raise Exception('The hamiltonian indices are out of range given J='+str(J)+', I='+str(I)+', dim(H)='+str(dim_H))
    
    mJ_1 = -J + index_1 // num_mI
    mI_1 = -I + index_1 % num_m
    
    mJ_2 = -J + index_2 // num_mI
    mI_2 = -I + index_2 % num_m
    
    H_contrib_J = 0
    if mJ_1 == mJ_2 and mI_1 == mI_2:
        H_contrib_J = B0 * J*(J + 1) * hbar * hbar 
    
    H_contrib_E = 0
    sign = 1 - 2 * ((1 - J - mI_2) % 2)
    if mI_1 == mI_2:
        H_contrib_E = dip * E * 2 * math.sqrt((2*J+1)*math.pi/3) * sign * py3nj.wigner3j(2*J, 2, 2*J, 
                                                                                         2*mJ_1, 0, -2*mJ_2)
    H_contrib_JI = 0
    sign1 = 1 - 2 * ((J+I-mJ_1-mI_1) % 2)    
    for p in [-1,0,1]:
        H_contrib_JI += sign1 * (1 - 2 * (p % 2)) * py3nj.wigner3j(2*J, 2, 2*J,
                                               -2*mJ_1, 2*p, 2*mJ_2) * py3nj.wigner3j(2*I, 2, 2*I, 
                                                                                    -2*mI_1, -2*p, 2*mI_2) * math.sqrt(J*(J+1)*(2*J+1)*I*(I+1)*(2*I+1))
    H_contrib_JI *= C_I
    
    return H_contrib_JI + H_contrib_E + H_contrib_J
    

In [46]:
def findH(E, J=1, I=1, B0=1, hbar=1, dip=1, C_I=1):
    num_m = 2*J+1
    num_mI = 2*I+1
    dim_H = num_m * num_mI
    
    # Initialize Hamiltonian matrix
    H = np.zeros((dim_H,dim_H))
    
    for index_1 in range(dim_H):
        for index_2 in range(dim_H):
            H[index_1, index_2] = getHij(index_1, index_2, E=E, J=J, I=I, B0=B0, hbar=hbar, dip=dip, C_I=C_I)
    
    return H

In [47]:
def get_eigenstates_for_E_range(H, E_min, E_max, E_step):
  # This method returns a dictionary which will contain dim arrays within it, each of
  # which contains the (eigenvalue, eigenvector) tuples corresponding to an
  # electric field value.
  
  Efield_eigenstates = {}
  for E in range(E_min,E_max,E_step):
    eigenstates = scipy.linalg.eig(H)
    eigenstates = zip(np.real(eigenstates[0]), eigenstates[1])
    Efield_eigenstates['E='+str(E)] = (sorted(eigenstates, key=lambda x: (x[0])))
  return Efield_eigenstates

In [62]:
H = findH(1,1,1,1,1,1,0)

In [63]:
print(H)

[[0.55279749 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         3.44720251 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.55279749 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         2.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         2.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         2.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  3.44720251 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.55279749 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         3.44720251]]
